# Load Image with labels from IDR, re-analyze using StarDist

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ome/EMBL-EBI-imaging-course-05-2023/blob/main/Day_4/stardist.ipynb)


The notebook shows how to load an IDR image with labels.

The image is referenced in the paper "NesSys: a novel method for accurate nuclear segmentation in 3D" published August 2019 in PLOS Biology: https://doi.org/10.1371/journal.pbio.3000388 and can be viewed online in the [Image Data Resource](https://idr.openmicroscopy.org/webclient/?show=image-6001247).


In this notebook, the image is loaded together with the labels and analyzed using [StarDist](https://github.com/stardist/stardist). The StarDist analysis produces a segmentation, which is then viewed side-by-side with the original segmentations produced by the authors of the paper obtained via the loaded labels.

If you wish to run the notebook locally or run the corresponding [Python script](../scripts/idr0062_prediction.py), please read instruction in [README](https://github.com/ome/omero-guide-python/blob/master/README.md).

## Summary:
![Overview](./includes/StarDistIDR.png)

## Launch

This notebook uses the [environment_stardist.yml](./environment_stardist.yml) file.

See [Setup](./workshop.ipynb).

### Install dependencies if required

The cell below will install dependencies if you choose to run the notebook in [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true). 

In [2]:
%pip install omero-py==5.13.1 stardist==0.8.2 geojson==3.0.1 omero-cli-zarr==0.5.2

### Import 

In [14]:
from omero.gateway import BlitzGateway

import matplotlib.pyplot as plt
%matplotlib inline
import numpy

### Create a connection to IDR

In [15]:
HOST = 'ws://idr.openmicroscopy.org/omero-ws'
conn = BlitzGateway('public', 'public',
                    host=HOST, secure=True)
print(conn.connect())
conn.c.enableKeepAlive(60)

True


### IDR image to analyze

In [16]:
image_id = 6001247

In [17]:
image = conn.getObject("Image", image_id)

### Helper method to load the 5D image

The image is loaded a TCZYX numpy array.

In [18]:
def load_numpy_array(image):
    pixels = image.getPrimaryPixels()
    size_z = image.getSizeZ()
    size_c = image.getSizeC()
    size_t = image.getSizeT()
    size_y = image.getSizeY()
    size_x = image.getSizeX()
    z, t, c = 0, 0, 0  # first plane of the image

    zct_list = []
    for t in range(size_t):
        for c in range(size_c):  # all channels
            for z in range(size_z):  # get the Z-stack
                zct_list.append((z, c, t))

    values = []
    # Load all the planes as YX numpy array
    planes = pixels.getPlanes(zct_list)
    s = "t:%s c:%s z:%s y:%s x:%s" % (size_t, size_c, size_z, size_y, size_x)
    print(s)
    print("Downloading image %s" % image.getName())
    all_planes = numpy.stack(list(planes))
    shape = (size_t, size_c, size_z, size_y, size_x)
    return numpy.reshape(all_planes, newshape=shape)

### Load the binary data
Load the binary data as a numpy array

In [19]:
data = load_numpy_array(image)

t:1 c:2 z:257 y:210 x:253


## Load StarDist trained model 

In [20]:
from stardist.models import StarDist2D
model_versatile = StarDist2D.from_pretrained('2D_demo')

Found model '2D_demo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.486166, nms_thresh=0.5.


## Prediction based on a default StarDist model
Normalize the input image

``model_versatile.predict_instances`` will

 * predict object probabilities and star-convex polygon distances (see model.predict if you want those)
 * perform non-maximum suppression (with overlap threshold nms_thresh) for polygons above object probability threshold prob_thresh.
 * render all remaining polygon instances in a label image
 * return the label instances image and also the details (coordinates, etc.) of all remaining polygons

In [21]:
from csbdeep.utils import normalize
axis_norm = (0,1)
c = 1
img = normalize(data[0, c, :, :, :], 1,99.8, axis=axis_norm)
results = []
results_details = []
for i in range(len(img)):
    new_labels, details = model_versatile.predict_instances(img[i])
    results_details.append(details)
    results.append(new_labels)

label_slices = numpy.array(results)

1/1 [==============================] - 0s 54ms/step


## Load the original labels
Load the original labels in order to compare them with the StarDist ones
Original labels have been saved as mask.

In [22]:
from omero_zarr import masks

In [23]:
roi_service = conn.getRoiService()
result = roi_service.findByImage(image_id, None)

dims = (image.getSizeT(), image.getSizeC(), image.getSizeZ(), image.getSizeY(), image.getSizeX())
shapes = []
for roi in result.rois:
    shapes.append(roi.copyShapes())

saver = masks.MaskSaver(None, image, numpy.int64)
labels, fillColors, properties = saver.masks_to_labels(shapes, mask_shape=dims)

In [24]:
print(labels.shape)

(1, 2, 257, 210, 253)


## Compare labels
Display the original labels and the labels based on StarDist prediction side-by-side

In [ ]:
from ipywidgets import *

def update(z=0):
    c = 1
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(data[0, c, z, :, :], cmap='jet')
    try:
        plt.imshow(labels[0, c, z, :, :], cmap='gray', alpha=0.5)
    except Exception:
        print(z)
    plt.subplot(122)
    plt.imshow(data[0, c, z, :, :], cmap='gray')
    plt.imshow(label_slices[z, :, :], cmap='jet', alpha=0.5)
    plt.tight_layout()
    fig.canvas.flush_events()

interact(update, z= widgets.IntSlider(value=1, min=0, max=data.shape[2]-1, step=1, description="Select Z", continuous_update=False))

### Close the connection  

In [13]:
conn.close()

## Save the StarDist labels

StarDist offers method to save the labels into ImageJ rois using ``export_imagej_rois``. This is outside the scope of this notebook. 

Below we show how to save the segmentation represented as polygon details locally in a machine- and human-readable format: **geojson**.

* Convert the StarDist polygon coordinates into geojson Polygons
* Save the output in the `notebooks` folder in a `.geojson` file.

In [16]:
# Convert into Polygon and add to Geometry Collection
from geojson import Feature, FeatureCollection, Polygon
c = 1
shapes = []
for i in range(len(results_details)):
    details = results_details[i]
    for obj_id, region in enumerate(details['coord']):
        coordinates = []
        x = region[1]
        y = region[0]
        for j in range(len(x)):
            coordinates.append((float(x[j]), float(y[j])))
        # append the first coordinate to close the polygon
        coordinates.append(coordinates[0])
        shape = Polygon(coordinates)
        properties = {
            "stroke-width": 1,
            "z": i,
            "c": c,
        }
        shapes.append(Feature(geometry=shape, properties=properties))    

gc = FeatureCollection(shapes)

In [17]:
# Save the shapes as geojson
import geojson
geojson_file = "stardist_shapes_%s.geojson" % image_id
geojson_dump = geojson.dumps(gc, sort_keys=True)
with open(geojson_file, 'w') as out:
    out.write(geojson_dump)

### License (BSD 2-Clause)
Copyright (C) 2023 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. 